In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# load data set
mnist = input_data.read_data_sets('MNIST', one_hot=True)

# define global variable fot configurations
n_inputs = 28 # 输入一行，一行有28个数据
max_time = 28 # 一共28行
lstm_size = 100 # 隐含层单元
n_classes = 10 # 10个分类
batch_size= 50 # 每个批次共有50个样本
n_batch = mnist.train.num_examples // batch_size # 共有的批次数

# 这里的None表示任意的长度
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# 初始化权值和偏移值
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))

# 定义RNN网络
def RNN(X, Weights, biases):
    # inputs=[batch_size, maz_time, n_inputs]
    inputs = tf.reshape(X, [-1, max_time, n_inputs])
    # 定义LSTM基本CELL 
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    # final_state[0]是cell_state
    # final_state[1]是hidden_state
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1], Weights) + biases)
    return results

# 计算RNN的返回结果
prediction = RNN(x, weights, biases)
# loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
# adam optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 结果存放在一个布尔变量中
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
# 计算准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# 初始化
init =  tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={
                x: batch_xs,
                y: batch_ys,               
            })
        acc = sess.run(accuracy, feed_dict={
            x: mnist.test.images,
            y: mnist.test.labels,
        })
        print("Iter: " + str(epoch) + ", Testing Accuracy: " + str(acc))
    

    

Extracting MNIST\train-images-idx3-ubyte.gz
Extracting MNIST\train-labels-idx1-ubyte.gz
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz
Iter: 0, Testing Accuracy: 0.7369
